In [54]:
import mouse
import cv2 as cv
import numpy as np
import mediapipe as mp
import pandas as pd
import numpy as np
import os
import torch
import torch.nn.functional as F 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [55]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers=nn.Sequential(
            nn.Linear(63,512),
            nn.ReLU(),
            nn.Linear(512,364),
            nn.ReLU(),
            nn.Linear(364,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(64,6),
        )
    def forward(self,x):
        logits=self.layers(x)

        return logits


In [56]:
from torch.utils.data import Dataset
class gestureCreator(Dataset):
    def __init__(self,dataframe):
        self.data=dataframe
    def __len__(self):
        return len(self.data)
    def __getitem__(self):
        features=self.data.values.astype(float) 
        features=torch.tensor(features,dtype=torch.float32)
        return features   

In [57]:
m=torch.load('model_f.pt')
m.eval()
device = torch.device('cpu')


In [58]:
print(device)

cpu


In [59]:
gesture_mapping={
    0:'call',
    1:'dislike',
    2:'like',
    3:'right',
    4:'left',
    5:'stop'
}

In [61]:
mp_h=mp.solutions.hands
hand=mp_h.Hands()
mp_draw=mp.solutions.drawing_utils
cap=cv.VideoCapture(1)
while True:

    ret,frame=cap.read()
    cv.flip(frame,1)

    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    result = hand.process(frame_rgb)

    if result.multi_hand_landmarks:
        for hand_index, hand_landmarks in enumerate(result.multi_hand_landmarks):
            handedness = result.multi_handedness[hand_index].classification[0].label
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_h.HAND_CONNECTIONS)
            land = []

            for lm in hand_landmarks.landmark:
                land.extend([lm.x, lm.y, lm.z])
               
            
            features=torch.tensor(land,dtype=torch.float32).unsqueeze(0).to(device)
            print(features)
            with torch.no_grad():
                out=m(features)
                _,val=torch.max(out,1)
                #print("out",val)
                pred=val.item()
                label=gesture_mapping.get(pred,"Unknown")
                print("label",label)
            wrist_pos=hand_landmarks.landmark[0]
            h,w,_=frame.shape
            wrist_x=int(wrist_pos.x*w)
            wrist_y=int(wrist_pos.y*h)
            pos=(wrist_x,wrist_y-30)

            cv.putText(
                frame, 
                f'{label}', 
                pos, 
                cv.FONT_HERSHEY_SIMPLEX, 
                1, 
                (255, 0, 0), 
                2, 
                cv.LINE_AA
            )

    # Display the image 
    cv.imshow('feed', cv.flip(frame, 1))
    if cv.waitKey(1) == ord("q"):
        cap.release()

        cv.destroyAllWindows()
        break

tensor([[ 6.0274e-01,  9.4242e-01,  1.0312e-07,  6.1012e-01,  9.1646e-01,
         -1.4490e-02,  6.1675e-01,  8.7592e-01, -1.8974e-02,  6.1838e-01,
          8.4075e-01, -2.2740e-02,  6.1373e-01,  8.1728e-01, -2.5685e-02,
          6.0378e-01,  8.1271e-01, -4.7691e-03,  6.0543e-01,  7.5773e-01,
         -1.3418e-02,  6.0810e-01,  7.2672e-01, -2.1489e-02,  6.1056e-01,
          7.0096e-01, -2.7301e-02,  6.0890e-01,  8.0908e-01, -4.6551e-03,
          6.1035e-01,  7.5000e-01, -1.2685e-02,  6.1298e-01,  7.1441e-01,
         -2.0873e-02,  6.1489e-01,  6.8571e-01, -2.6368e-02,  6.1555e-01,
          8.1722e-01, -6.9622e-03,  6.1909e-01,  7.6240e-01, -1.6508e-02,
          6.2236e-01,  7.3139e-01, -2.3376e-02,  6.2444e-01,  7.0701e-01,
         -2.7365e-02,  6.2376e-01,  8.3386e-01, -1.0848e-02,  6.3494e-01,
          7.9836e-01, -1.8202e-02,  6.4216e-01,  7.7711e-01, -2.0867e-02,
          6.4837e-01,  7.6002e-01, -2.2357e-02]])
label stop
tensor([[ 6.4244e-01,  7.4893e-01,  5.5622e-09,  6.